In [2]:
import pandas as pd
import numpy as np
import bilby
import lal
import gwpy
from gwpy.timeseries import TimeSeries
import matplotlib.pyplot as plt
import copy

import sys
sys.path.append("..")


from event_table import call_event_table
from waveforms import osc_freq_XPHM, mem_freq_XPHM, mem_freq_XPHM_only
from create_post_dict import create_post_dict, extract_relevant_info, process_bilby_result
from reweight_mem_parallel import call_data_GWOSC

In [7]:
event_number = 8
event_name, file_path, trigger_time, duration, waveform, data_file = call_event_table()[event_number]
samples, meta_dict, config_dict, priors, psds, calibration = create_post_dict(file_path, waveform)
args = extract_relevant_info(meta_dict, config_dict)

logger = bilby.core.utils.logger

sampling_frequency = args['sampling_frequency']
maximum_frequency = args['maximum_frequency']
minimum_frequency = args['minimum_frequency']
reference_frequency = args['reference_frequency']
roll_off = args['tukey_roll_off']
duration = args['duration']
post_trigger_duration = args['post_trigger_duration']
trigger_time = args['trigger_time']
detectors = args['detectors']

print('information')
print('event name: ', event_name)
print('detectors: ', detectors)

detectors = args['detectors']
if 'V1' in detectors:
    detectors.remove('V1')

end_time = trigger_time + post_trigger_duration
start_time = end_time - duration

psd_duration = 32*duration # deprecated
psd_start_time = start_time - psd_duration # deprecated
psd_end_time = start_time # deprecated

waveform_name = args['waveform_approximant']

ifo_list= call_data_GWOSC(logger, args, 
                        calibration, samples, detectors,
                        start_time, end_time, 
                        psd_start_time, psd_end_time, 
                        duration, sampling_frequency, 
                        roll_off, minimum_frequency, maximum_frequency,
                        psds_array=psds)

priors2 = copy.copy(priors)

# test if bilby oscillatory waveform = gwmemory oscillatory waveform.
waveform_generator_osc = bilby.gw.waveform_generator.WaveformGenerator(
    duration=duration,
    sampling_frequency=sampling_frequency,
    frequency_domain_source_model= bilby.gw.source.lal_binary_black_hole,
    parameter_conversion = bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters,
    waveform_arguments=dict(duration=duration,
                            minimum_frequency=minimum_frequency,
                            maximum_frequency=maximum_frequency,
                            sampling_frequency=sampling_frequency,
                            reference_frequency=reference_frequency,
                            waveform_approximant=waveform_name,
                            )

)

target_likelihood = bilby.gw.likelihood.GravitationalWaveTransient(
    ifo_list,
    waveform_generator_osc,
    time_marginalization = True,
    distance_marginalization = True,
    distance_marginalization_lookup_table = '../TD.npz',
    jitter_time=True,
    priors = priors2,
    reference_frame = args['reference_frame'],
    time_reference = args['time_reference'],
)
print('stored log likelihood', '| calculated log likelihood')
for i in range(10):
    posterior = samples.iloc[i].to_dict()
    target_likelihood.parameters.update(posterior)
    reference_parameters = dict(luminosity_distance = priors2['luminosity_distance'],
                                geocent_time = priors2['geocent_time'])
    target_likelihood.parameters.update(reference_parameters)
    likelihood = target_likelihood.log_likelihood_ratio()
    print(samples['log_likelihood'].iloc[i],likelihood)

18:53 bilby INFO    : Downloading analysis data for ifo L1


Jitter time setting cannot be found. Use default setting.
channel_dict {'H1': 'DCH-CLEAN_STRAIN_C02', 'L1': 'DCH-CLEAN_STRAIN_C02', 'V1': 'Hrec_hoft_V1O2Repro2A_16384Hz'}
['', 'home', 'daniel.williams', 'events', 'O3', 'o3a_final', 'run_directories', '', 'GW170818', 'ProdF7']
information
event name:  GW170818
detectors:  ['L1', 'H1', 'V1']


18:53 bilby INFO    : Downloading analysis data for ifo H1


Using pre-computed psd from results file


18:54 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_black_hole
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters


Using pre-computed psd from results file


18:54 bilby INFO    : Loaded distance marginalisation lookup table from ../TD.npz.


stored log likelihood | calculated log likelihood
61.49206647861965 51.66951311123018
61.10075751046299 50.818156072200836
59.12672625501739 51.75056994749835
55.96990766273144 47.115619363131046
59.577235624802576 50.29536930790279
57.569143395324296 48.618799553349916
60.33188150841454 50.724406029389485
61.742284236457905 52.354260116113196
57.75200697712786 48.36696375689809
60.347512814011296 51.55224511701354


In [7]:
event_number = 0
event_name, file_path, trigger_time, duration, waveform, data_file = call_event_table()[event_number]
samples, meta_dict, config_dict, priors, psds, calibration = create_post_dict(file_path, waveform)
args = extract_relevant_info(meta_dict, config_dict)

logger = bilby.core.utils.logger

sampling_frequency = args['sampling_frequency']
maximum_frequency = args['maximum_frequency']
minimum_frequency = args['minimum_frequency']
reference_frequency = args['reference_frequency']
roll_off = args['tukey_roll_off']
duration = args['duration']
post_trigger_duration = args['post_trigger_duration']
trigger_time = args['trigger_time']
detectors = args['detectors']

print('information')
print('event name: ', event_name)
print('detectors: ', detectors)

detectors = args['detectors']
if 'V1' in detectors:
    detectors.remove('V1')

end_time = trigger_time + post_trigger_duration
start_time = end_time - duration

psd_duration = 32*duration # deprecated
psd_start_time = start_time - psd_duration # deprecated
psd_end_time = start_time # deprecated

waveform_name = args['waveform_approximant']

ifo_list= call_data_GWOSC(logger, args, 
                        calibration, samples, detectors,
                        start_time, end_time, 
                        psd_start_time, psd_end_time, 
                        duration, sampling_frequency, 
                        roll_off, minimum_frequency, maximum_frequency,
                        psds_array=psds)

priors2 = copy.copy(priors)

# test if bilby oscillatory waveform = gwmemory oscillatory waveform.
waveform_generator_osc = bilby.gw.waveform_generator.WaveformGenerator(
    duration=duration,
    sampling_frequency=sampling_frequency,
    frequency_domain_source_model= bilby.gw.source.lal_binary_black_hole,
    parameter_conversion = bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters,
    waveform_arguments=dict(duration=duration,
                            minimum_frequency=minimum_frequency,
                            maximum_frequency=maximum_frequency,
                            sampling_frequency=sampling_frequency,
                            reference_frequency=reference_frequency,
                            waveform_approximant=waveform_name,
                            )

)

target_likelihood = bilby.gw.likelihood.GravitationalWaveTransient(
    ifo_list,
    waveform_generator_osc,
    time_marginalization = True,
    distance_marginalization = True,
    distance_marginalization_lookup_table = '../TD.npz',
    jitter_time=True,
    priors = priors2,
    reference_frame = args['reference_frame'],
    time_reference = args['time_reference'],
)

for i in range(10):
    posterior = samples.iloc[i].to_dict()
    target_likelihood.parameters.update(posterior)
    reference_parameters = dict(luminosity_distance = priors2['luminosity_distance'],
                                geocent_time = priors2['geocent_time'])
    target_likelihood.parameters.update(reference_parameters)
    likelihood = target_likelihood.log_likelihood_ratio()
    print(samples['log_likelihood'].iloc[i],likelihood)

18:42 bilby INFO    : Downloading analysis data for ifo H1


Jitter time setting cannot be found. Use default setting.
channel_dict {'H1': 'DCS-CALIB_STRAIN_C02', 'L1': 'DCS-CALIB_STRAIN_C02'}
['', 'home', 'daniel.williams', 'events', 'O3', 'o3a', 'run_directories', 'GW150914', 'ProdF4']
information
event name:  GW150914
detectors:  ['H1', 'L1']


18:42 bilby INFO    : Downloading analysis data for ifo L1


Using pre-computed psd from results file


18:42 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_black_hole
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters


Using pre-computed psd from results file


18:42 bilby INFO    : Loaded distance marginalisation lookup table does not match for distance_array.
/home/shunyin.cheung/.conda/envs/altenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
18:42 bilby INFO    : Building lookup table for distance marginalisation.
100%|██████████| 400/400 [01:16<00:00,  5.19it/s]


322.30427095284983 321.3882985283915
320.30148251901113 320.28737195178644
320.65400935312505 320.8598700799437
324.2656609494165 323.70300767492586
322.7294264130223 323.6245941174861
325.55371356766864 324.2305526286645
321.13250028898017 319.8295930422306
321.76573539575816 321.80137512208023
320.6351240330521 320.4791509033959
322.0405694919023 321.86030410178284


In [8]:
event_number = 1
event_name, file_path, trigger_time, duration, waveform, data_file = call_event_table()[event_number]
samples, meta_dict, config_dict, priors, psds, calibration = create_post_dict(file_path, waveform)
args = extract_relevant_info(meta_dict, config_dict)

logger = bilby.core.utils.logger

sampling_frequency = args['sampling_frequency']
maximum_frequency = args['maximum_frequency']
minimum_frequency = args['minimum_frequency']
reference_frequency = args['reference_frequency']
roll_off = args['tukey_roll_off']
duration = args['duration']
post_trigger_duration = args['post_trigger_duration']
trigger_time = args['trigger_time']
detectors = args['detectors']

print('information')
print('event name: ', event_name)
print('detectors: ', detectors)

detectors = args['detectors']
if 'V1' in detectors:
    detectors.remove('V1')

end_time = trigger_time + post_trigger_duration
start_time = end_time - duration

psd_duration = 32*duration # deprecated
psd_start_time = start_time - psd_duration # deprecated
psd_end_time = start_time # deprecated

waveform_name = args['waveform_approximant']

ifo_list= call_data_GWOSC(logger, args, 
                        calibration, samples, detectors,
                        start_time, end_time, 
                        psd_start_time, psd_end_time, 
                        duration, sampling_frequency, 
                        roll_off, minimum_frequency, maximum_frequency,
                        psds_array=psds)

priors2 = copy.copy(priors)

# test if bilby oscillatory waveform = gwmemory oscillatory waveform.
waveform_generator_osc = bilby.gw.waveform_generator.WaveformGenerator(
    duration=duration,
    sampling_frequency=sampling_frequency,
    frequency_domain_source_model= bilby.gw.source.lal_binary_black_hole,
    parameter_conversion = bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters,
    waveform_arguments=dict(duration=duration,
                            minimum_frequency=minimum_frequency,
                            maximum_frequency=maximum_frequency,
                            sampling_frequency=sampling_frequency,
                            reference_frequency=reference_frequency,
                            waveform_approximant=waveform_name,
                            )

)

target_likelihood = bilby.gw.likelihood.GravitationalWaveTransient(
    ifo_list,
    waveform_generator_osc,
    time_marginalization = True,
    distance_marginalization = True,
    distance_marginalization_lookup_table = '../TD.npz',
    jitter_time=True,
    priors = priors2,
    reference_frame = args['reference_frame'],
    time_reference = args['time_reference'],
)

for i in range(10):
    posterior = samples.iloc[i].to_dict()
    target_likelihood.parameters.update(posterior)
    reference_parameters = dict(luminosity_distance = priors2['luminosity_distance'],
                                geocent_time = priors2['geocent_time'])
    target_likelihood.parameters.update(reference_parameters)
    likelihood = target_likelihood.log_likelihood_ratio()
    print(samples['log_likelihood'].iloc[i],likelihood)

18:44 bilby INFO    : Downloading analysis data for ifo H1


Jitter time setting cannot be found. Use default setting.
channel_dict {'H1': 'DCS-CALIB_STRAIN_C02', 'L1': 'DCS-CALIB_STRAIN_C02'}
['', 'home', 'daniel.williams', 'events', 'O3', 'o3a_final', 'run_directories', '', 'GW151012', 'ProdF4']
information
event name:  GW151012
detectors:  ['H1', 'L1']


18:46 bilby INFO    : Downloading analysis data for ifo L1


Using pre-computed psd from results file


18:46 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_black_hole
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters


Using pre-computed psd from results file


18:46 bilby INFO    : Loaded distance marginalisation lookup table from ../TD.npz.


31.294287745639195 31.133934618522535
33.26265470613678 32.825632860828804
30.819025218309562 31.052557885005797
24.716952764892625 24.345583198511477
32.393754104883904 31.912141751210438
30.944337874712037 31.368248974951726
30.128299133647268 31.03316144591897
27.183242748646073 27.0430921983005
28.780401296466707 28.370569473367688
31.05841032444647 31.014553494415132


In [4]:
event_number = 3
event_name, file_path, trigger_time, duration, waveform, data_file = call_event_table()[event_number]
samples, meta_dict, config_dict, priors, psds, calibration = create_post_dict(file_path, waveform)
args = extract_relevant_info(meta_dict, config_dict)

logger = bilby.core.utils.logger

sampling_frequency = args['sampling_frequency']
maximum_frequency = args['maximum_frequency']
minimum_frequency = args['minimum_frequency']
reference_frequency = args['reference_frequency']
roll_off = args['tukey_roll_off']
duration = args['duration']
post_trigger_duration = args['post_trigger_duration']
trigger_time = args['trigger_time']
detectors = args['detectors']

print('information')
print('event name: ', event_name)
print('detectors: ', detectors)

detectors = args['detectors']
if 'V1' in detectors:
    detectors.remove('V1')

end_time = trigger_time + post_trigger_duration
start_time = end_time - duration

psd_duration = 32*duration # deprecated
psd_start_time = start_time - psd_duration # deprecated
psd_end_time = start_time # deprecated

waveform_name = args['waveform_approximant']

ifo_list= call_data_GWOSC(logger, args, 
                        calibration, samples, detectors,
                        start_time, end_time, 
                        psd_start_time, psd_end_time, 
                        duration, sampling_frequency, 
                        roll_off, minimum_frequency, maximum_frequency,
                        psds_array=psds)

priors2 = copy.copy(priors)

# test if bilby oscillatory waveform = gwmemory oscillatory waveform.
waveform_generator_osc = bilby.gw.waveform_generator.WaveformGenerator(
    duration=duration,
    sampling_frequency=sampling_frequency,
    frequency_domain_source_model= bilby.gw.source.lal_binary_black_hole,
    parameter_conversion = bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters,
    waveform_arguments=dict(duration=duration,
                            minimum_frequency=minimum_frequency,
                            maximum_frequency=maximum_frequency,
                            sampling_frequency=sampling_frequency,
                            reference_frequency=reference_frequency,
                            waveform_approximant=waveform_name,
                            )

)

target_likelihood = bilby.gw.likelihood.GravitationalWaveTransient(
    ifo_list,
    waveform_generator_osc,
    time_marginalization = True,
    distance_marginalization = True,
    distance_marginalization_lookup_table = '../TD.npz',
    jitter_time=True,
    priors = priors2,
    reference_frame = args['reference_frame'],
    time_reference = args['time_reference'],
)

for i in range(10):
    posterior = samples.iloc[i].to_dict()
    target_likelihood.parameters.update(posterior)
    reference_parameters = dict(luminosity_distance = priors2['luminosity_distance'],
                                geocent_time = priors2['geocent_time'])
    target_likelihood.parameters.update(reference_parameters)
    likelihood = target_likelihood.log_likelihood_ratio()
    print(samples['log_likelihood'].iloc[i],likelihood)

19:13 bilby INFO    : Downloading analysis data for ifo H1


Jitter time setting cannot be found. Use default setting.
channel_dict {'H1': 'DCH-CLEAN_STRAIN_C02', 'L1': 'DCH-CLEAN_STRAIN_C02'}
['', 'home', 'daniel.williams', 'events', 'O3', 'o3a_final', 'run_directories', '', 'GW170104', 'ProdF4']
information
event name:  GW170104
detectors:  ['H1', 'L1']


19:13 bilby INFO    : Downloading analysis data for ifo L1


Using pre-computed psd from results file


19:14 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_black_hole
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters


Using pre-computed psd from results file


19:14 bilby INFO    : Loaded distance marginalisation lookup table from ../TD.npz.


84.3138201496587 84.36057414395954
83.167606317808 83.26564015430378
84.00056429167402 83.94490652328467
85.3006907088475 85.03731137660829
84.09727649328131 84.31594884505964
82.04036522969737 82.13741460228083
83.80935751451489 83.81212695199832
82.97480345152786 82.98299972108165
83.28110581106311 83.74147378536357
83.49142161876397 83.64343956440973


In [5]:
event_number = 5
event_name, file_path, trigger_time, duration, waveform, data_file = call_event_table()[event_number]
samples, meta_dict, config_dict, priors, psds, calibration = create_post_dict(file_path, waveform)
args = extract_relevant_info(meta_dict, config_dict)

logger = bilby.core.utils.logger

sampling_frequency = args['sampling_frequency']
maximum_frequency = args['maximum_frequency']
minimum_frequency = args['minimum_frequency']
reference_frequency = args['reference_frequency']
roll_off = args['tukey_roll_off']
duration = args['duration']
post_trigger_duration = args['post_trigger_duration']
trigger_time = args['trigger_time']
detectors = args['detectors']

print('information')
print('event name: ', event_name)
print('detectors: ', detectors)

detectors = args['detectors']
if 'V1' in detectors:
    detectors.remove('V1')

end_time = trigger_time + post_trigger_duration
start_time = end_time - duration

psd_duration = 32*duration # deprecated
psd_start_time = start_time - psd_duration # deprecated
psd_end_time = start_time # deprecated

waveform_name = args['waveform_approximant']

ifo_list= call_data_GWOSC(logger, args, 
                        calibration, samples, detectors,
                        start_time, end_time, 
                        psd_start_time, psd_end_time, 
                        duration, sampling_frequency, 
                        roll_off, minimum_frequency, maximum_frequency,
                        psds_array=psds)

priors2 = copy.copy(priors)

# test if bilby oscillatory waveform = gwmemory oscillatory waveform.
waveform_generator_osc = bilby.gw.waveform_generator.WaveformGenerator(
    duration=duration,
    sampling_frequency=sampling_frequency,
    frequency_domain_source_model= bilby.gw.source.lal_binary_black_hole,
    parameter_conversion = bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters,
    waveform_arguments=dict(duration=duration,
                            minimum_frequency=minimum_frequency,
                            maximum_frequency=maximum_frequency,
                            sampling_frequency=sampling_frequency,
                            reference_frequency=reference_frequency,
                            waveform_approximant=waveform_name,
                            )

)

target_likelihood = bilby.gw.likelihood.GravitationalWaveTransient(
    ifo_list,
    waveform_generator_osc,
    time_marginalization = True,
    distance_marginalization = True,
    distance_marginalization_lookup_table = '../TD.npz',
    jitter_time=True,
    priors = priors2,
    reference_frame = args['reference_frame'],
    time_reference = args['time_reference'],
)

for i in range(10):
    posterior = samples.iloc[i].to_dict()
    target_likelihood.parameters.update(posterior)
    reference_parameters = dict(luminosity_distance = priors2['luminosity_distance'],
                                geocent_time = priors2['geocent_time'])
    target_likelihood.parameters.update(reference_parameters)
    likelihood = target_likelihood.log_likelihood_ratio()
    print(samples['log_likelihood'].iloc[i],likelihood)

19:16 bilby INFO    : Downloading analysis data for ifo H1


Jitter time setting cannot be found. Use default setting.
channel_dict {'H1': 'DCH-CLEAN_STRAIN_C02', 'L1': 'DCH-CLEAN_STRAIN_C02', 'V1': 'Hrec_hoft_V1O2Repro2A_16384Hz'}
['', 'home', 'daniel.williams', 'events', 'O3', 'o3a_final', 'run_directories', '', 'GW170729', 'ProdF10']
information
event name:  GW170729
detectors:  ['H1', 'L1', 'V1']


19:16 bilby INFO    : Downloading analysis data for ifo L1


Using pre-computed psd from results file


19:16 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_black_hole
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters
19:16 bilby INFO    : Loaded distance marginalisation lookup table from ../TD.npz.


Using pre-computed psd from results file
43.09752838383797 41.32865422182828
38.733543490410014 36.4932821571847
48.99378571457131 47.71714232139952
47.829432209892076 48.179876751760865
41.38606221259258 38.64679020155381
48.193407316089484 46.3122135186991
50.46963718404028 48.448524394257376
45.772241335356966 46.10887929894709
46.082125143673586 46.26278311651207
43.42179530884173 44.44488574282893


In [2]:
event_number = 70
event_name, file_path, trigger_time, duration, waveform, data_file = call_event_table()[event_number]
samples, meta_dict, config_dict, priors, psds, calibration = create_post_dict(file_path, waveform)
args = extract_relevant_info(meta_dict, config_dict)

logger = bilby.core.utils.logger

sampling_frequency = args['sampling_frequency']
maximum_frequency = args['maximum_frequency']
minimum_frequency = args['minimum_frequency']
reference_frequency = args['reference_frequency']
roll_off = args['tukey_roll_off']
duration = args['duration']
post_trigger_duration = args['post_trigger_duration']
trigger_time = args['trigger_time']
detectors = args['detectors']

print('information')
print('event name: ', event_name)
print('detectors: ', detectors)

detectors = args['detectors']
if 'V1' in detectors:
    detectors.remove('V1')

end_time = trigger_time + post_trigger_duration
start_time = end_time - duration

psd_duration = 32*duration # deprecated
psd_start_time = start_time - psd_duration # deprecated
psd_end_time = start_time # deprecated

waveform_name = args['waveform_approximant']

ifo_list= call_data_GWOSC(logger, args, 
                        calibration, samples, detectors,
                        start_time, end_time, 
                        psd_start_time, psd_end_time, 
                        duration, sampling_frequency, 
                        roll_off, minimum_frequency, maximum_frequency,
                        psds_array=psds)

priors2 = copy.copy(priors)

# test if bilby oscillatory waveform = gwmemory oscillatory waveform.
waveform_generator_osc = bilby.gw.waveform_generator.WaveformGenerator(
    duration=duration,
    sampling_frequency=sampling_frequency,
    frequency_domain_source_model= bilby.gw.source.lal_binary_black_hole,
    parameter_conversion = bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters,
    waveform_arguments=dict(duration=duration,
                            minimum_frequency=minimum_frequency,
                            maximum_frequency=maximum_frequency,
                            sampling_frequency=sampling_frequency,
                            reference_frequency=reference_frequency,
                            waveform_approximant=waveform_name,
                            )

)

target_likelihood = bilby.gw.likelihood.GravitationalWaveTransient(
    ifo_list,
    waveform_generator_osc,
    time_marginalization = True,
    distance_marginalization = True,
    distance_marginalization_lookup_table = '../TD.npz',
    jitter_time=True,
    priors = priors2,
    reference_frame = args['reference_frame'],
    time_reference = args['time_reference'],
)

for i in range(10):
    posterior = samples.iloc[i].to_dict()
    target_likelihood.parameters.update(posterior)
    reference_parameters = dict(luminosity_distance = priors2['luminosity_distance'],
                                geocent_time = priors2['geocent_time'])
    target_likelihood.parameters.update(reference_parameters)
    likelihood = target_likelihood.log_likelihood_ratio()
    print(samples['log_likelihood'].iloc[i],likelihood)

19:31 bilby INFO    : Downloading analysis data for ifo H1


Jitter time setting cannot be found. Use default setting.
channel_dict {'H1': 'DCS-CALIB_STRAIN_CLEAN_SUB60HZ_C01', 'L1': 'DCS-CALIB_STRAIN_CLEAN_SUB60HZ_C01'}
['', 'home', 'daniel.williams', 'events', 'O3', 'o3b', 'run_directories', 'S200128d', 'Prod6']
information
event name:  GW200128
detectors:  ['H1', 'L1']


19:31 bilby INFO    : Downloading analysis data for ifo L1


Using pre-computed psd from results file


19:31 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_black_hole
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters


Using pre-computed psd from results file


19:31 bilby INFO    : Loaded distance marginalisation lookup table from ../TD.npz.


46.454554025662695 45.37634129348023
48.914275648546365 49.486387465414104
50.32807187848023 50.46559509614716
51.815565992841464 51.912495899788276
47.02574737825925 47.02466605680508
49.066449703904446 49.641822438306264
50.856912189222626 50.67858727954246
46.58761895637697 47.46005212996484
49.769207658185174 49.52569829103165
51.46049302036471 51.65095950843296


In [3]:
event_number = 71
event_name, file_path, trigger_time, duration, waveform, data_file = call_event_table()[event_number]
samples, meta_dict, config_dict, priors, psds, calibration = create_post_dict(file_path, waveform)
args = extract_relevant_info(meta_dict, config_dict)

logger = bilby.core.utils.logger

sampling_frequency = args['sampling_frequency']
maximum_frequency = args['maximum_frequency']
minimum_frequency = args['minimum_frequency']
reference_frequency = args['reference_frequency']
roll_off = args['tukey_roll_off']
duration = args['duration']
post_trigger_duration = args['post_trigger_duration']
trigger_time = args['trigger_time']
detectors = args['detectors']

print('information')
print('event name: ', event_name)
print('detectors: ', detectors)

detectors = args['detectors']
if 'V1' in detectors:
    detectors.remove('V1')

end_time = trigger_time + post_trigger_duration
start_time = end_time - duration

psd_duration = 32*duration # deprecated
psd_start_time = start_time - psd_duration # deprecated
psd_end_time = start_time # deprecated

waveform_name = args['waveform_approximant']

ifo_list= call_data_GWOSC(logger, args, 
                        calibration, samples, detectors,
                        start_time, end_time, 
                        psd_start_time, psd_end_time, 
                        duration, sampling_frequency, 
                        roll_off, minimum_frequency, maximum_frequency,
                        psds_array=psds)

priors2 = copy.copy(priors)

# test if bilby oscillatory waveform = gwmemory oscillatory waveform.
waveform_generator_osc = bilby.gw.waveform_generator.WaveformGenerator(
    duration=duration,
    sampling_frequency=sampling_frequency,
    frequency_domain_source_model= bilby.gw.source.lal_binary_black_hole,
    parameter_conversion = bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters,
    waveform_arguments=dict(duration=duration,
                            minimum_frequency=minimum_frequency,
                            maximum_frequency=maximum_frequency,
                            sampling_frequency=sampling_frequency,
                            reference_frequency=reference_frequency,
                            waveform_approximant=waveform_name,
                            )

)

target_likelihood = bilby.gw.likelihood.GravitationalWaveTransient(
    ifo_list,
    waveform_generator_osc,
    time_marginalization = True,
    distance_marginalization = True,
    distance_marginalization_lookup_table = '../TD.npz',
    jitter_time=True,
    priors = priors2,
    reference_frame = args['reference_frame'],
    time_reference = args['time_reference'],
)


print('stored log likelihood | calculated log likelihood')
for i in range(10):
    posterior = samples.iloc[i].to_dict()
    target_likelihood.parameters.update(posterior)
    reference_parameters = dict(luminosity_distance = priors2['luminosity_distance'],
                                geocent_time = priors2['geocent_time'])
    target_likelihood.parameters.update(reference_parameters)
    likelihood = target_likelihood.log_likelihood_ratio()
    print(samples['log_likelihood'].iloc[i],likelihood)

19:10 bilby INFO    : Downloading analysis data for ifo H1


Jitter time setting cannot be found. Use default setting.
channel_dict {'H1': 'DCS-CALIB_STRAIN_CLEAN_SUB60HZ_C01', 'L1': 'DCS-CALIB_STRAIN_CLEAN_SUB60HZ_C01_P1800169_v4', 'V1': 'Hrec_hoft_16384Hz'}
['', 'home', 'daniel.williams', 'events', 'O3', 'o3b', 'run_directories', 'S200129a', 'Prod5']
information
event name:  GW200129
detectors:  ['H1', 'L1', 'V1']


19:10 bilby INFO    : Downloading analysis data for ifo L1


Using pre-computed psd from results file


19:10 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_black_hole
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters
19:10 bilby INFO    : Loaded distance marginalisation lookup table from ../TD.npz.


Using pre-computed psd from results file
stored log likelihood | calculated log likelihood
343.3227330124775 323.01450865688844
351.40100526024895 328.58757990628936
348.6071723056927 324.2286082327364
343.8096152410793 326.02915056913446
349.5071979302905 327.68828956636486
342.32545779665253 319.8435788528477
341.96529596978445 322.68936942101175
344.54033078610394 323.4948385240985
345.6460803216168 321.8244111797693
350.1555587581522 328.46734910922373


In [3]:
event_number = 11
event_name, file_path, trigger_time, duration, waveform, data_file = call_event_table()[event_number]
samples, meta_dict, config_dict, priors, psds, calibration = create_post_dict(file_path, waveform)
args = extract_relevant_info(meta_dict, config_dict)

logger = bilby.core.utils.logger

sampling_frequency = args['sampling_frequency']
maximum_frequency = args['maximum_frequency']
minimum_frequency = args['minimum_frequency']
reference_frequency = args['reference_frequency']
roll_off = args['tukey_roll_off']
duration = args['duration']
post_trigger_duration = args['post_trigger_duration']
trigger_time = args['trigger_time']
detectors = args['detectors']

print('information')
print('event name: ', event_name)
print('detectors: ', detectors)

detectors = args['detectors']
if 'V1' in detectors:
    detectors.remove('V1')

end_time = trigger_time + post_trigger_duration
start_time = end_time - duration

psd_duration = 32*duration # deprecated
psd_start_time = start_time - psd_duration # deprecated
psd_end_time = start_time # deprecated

waveform_name = args['waveform_approximant']

ifo_list= call_data_GWOSC(logger, args, 
                        calibration, samples, detectors,
                        start_time, end_time, 
                        psd_start_time, psd_end_time, 
                        duration, sampling_frequency, 
                        roll_off, minimum_frequency, maximum_frequency,
                        psds_array=psds)

priors2 = copy.copy(priors)

# test if bilby oscillatory waveform = gwmemory oscillatory waveform.
waveform_generator_osc = bilby.gw.waveform_generator.WaveformGenerator(
    duration=duration,
    sampling_frequency=sampling_frequency,
    frequency_domain_source_model= bilby.gw.source.lal_binary_black_hole,
    parameter_conversion = bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters,
    waveform_arguments=dict(duration=duration,
                            minimum_frequency=minimum_frequency,
                            maximum_frequency=maximum_frequency,
                            sampling_frequency=sampling_frequency,
                            reference_frequency=reference_frequency,
                            waveform_approximant=waveform_name,
                            )

)

target_likelihood = bilby.gw.likelihood.GravitationalWaveTransient(
    ifo_list,
    waveform_generator_osc,
    time_marginalization = True,
    distance_marginalization = True,
    distance_marginalization_lookup_table = '../TD.npz',
    jitter_time=True,
    priors = priors2,
    reference_frame = args['reference_frame'],
    time_reference = args['time_reference'],
)

print(samples.keys())

for i in range(10):
    posterior = samples.iloc[i].to_dict()
    target_likelihood.parameters.update(posterior)
    reference_parameters = dict(luminosity_distance = priors2['luminosity_distance'],
                                geocent_time = priors2['geocent_time'])
    target_likelihood.parameters.update(reference_parameters)
    likelihood = target_likelihood.log_likelihood_ratio()
    print(samples['log_likelihood'].iloc[i],likelihood)

18:42 bilby INFO    : Downloading analysis data for ifo H1


Jitter time setting cannot be found. Use default setting.
channel_dict {'H1': 'DCS-CALIB_STRAIN_CLEAN_SUB60HZ_C01', 'L1': 'DCS-CALIB_STRAIN_CLEAN_SUB60HZ_C01', 'V1': 'Hrec_hoft_16384Hz'}
['', 'home', 'daniel.williams', 'events', 'O3', 'o3a', 'run_directories', 'S190408a', 'ProdF6']
information
event name:  GW190408
detectors:  ['H1', 'L1', 'V1']


18:42 bilby INFO    : Downloading analysis data for ifo L1


Using pre-computed psd from results file


18:42 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_black_hole
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters
18:42 bilby INFO    : Loaded distance marginalisation lookup table from ../TD.npz.


Using pre-computed psd from results file
Index(['chirp_mass', 'mass_ratio', 'a_1', 'a_2', 'tilt_1', 'tilt_2', 'phi_12',
       'phi_jl', 'theta_jn', 'psi',
       ...
       'chi_p_2spin', 'spin_1z_infinity_only_prec_avg',
       'spin_2z_infinity_only_prec_avg', 'chi_eff_infinity_only_prec_avg',
       'chi_p_infinity_only_prec_avg', 'beta', 'psi_J',
       'cos_tilt_1_infinity_only_prec_avg',
       'cos_tilt_2_infinity_only_prec_avg', 'viewing_angle'],
      dtype='object', length=172)
96.13321557705225 95.68721276132023
97.52210499837784 96.47080420157651
95.39133885333096 95.21780949824928
95.0022100999799 93.59982160088657
95.29525480450286 94.11312100949
97.6515416670528 95.01613838827325
96.54191031546229 94.73761249040348
92.22958787420554 91.67718926971847
93.72577697255392 93.87061157515119
94.55299567374753 93.18565119494613


In [5]:
print(samples['V1_optimal_snr'])

0        0.709722
1        2.519534
2        0.747665
3        1.179041
4        2.507522
           ...   
53966    1.250815
53967    0.452119
53968    2.499889
53969    1.555109
53970    3.189811
Name: V1_optimal_snr, Length: 53971, dtype: float64


In [5]:
import pickle

file_path = '/home/shunyin.cheung/other_PE_run/GW190728.json'
data_file = '/home/shunyin.cheung/other_PE_run/data/GW190728_data_dump.pickle'


result = bilby.core.result.read_in_result(file_path)
samples = result.posterior
args = process_bilby_result(result.meta_data['command_line_args'])
priors = result.priors
priors2 = copy.copy(priors)
psds=None

with open(data_file, 'rb') as f:
    data_dump = pickle.load(f)
try:
    ifo_list = data_dump.interferometers

except AttributeError:
    ifo_list = data_dump['ifo_list']

sampling_frequency = ifo_list.sampling_frequency
maximum_frequency = args['maximum_frequency']
minimum_frequency = args['minimum_frequency']
reference_frequency = args['reference_frequency']
roll_off = args['tukey_roll_off']
duration = args['duration']

waveform_name = args['waveform_approximant']
    

# test if bilby oscillatory waveform = gwmemory oscillatory waveform.
waveform_generator_osc = bilby.gw.waveform_generator.WaveformGenerator(
    duration=duration,
    sampling_frequency=sampling_frequency,
    frequency_domain_source_model= bilby.gw.source.lal_binary_black_hole,
    parameter_conversion = bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters,
    waveform_arguments=dict(duration=duration,
                            minimum_frequency=minimum_frequency,
                            maximum_frequency=maximum_frequency,
                            sampling_frequency=sampling_frequency,
                            reference_frequency=reference_frequency,
                            waveform_approximant=waveform_name,
                            )

)

target_likelihood = bilby.gw.likelihood.GravitationalWaveTransient(
    ifo_list,
    waveform_generator_osc,
    time_marginalization = True,
    distance_marginalization = True,
    distance_marginalization_lookup_table = '../TD.npz',
    jitter_time=True,
    priors = priors2,
    reference_frame = args['reference_frame'],
    time_reference = args['time_reference'],
)

for i in range(10):
    posterior = samples.iloc[i].to_dict()
    if np.iscomplexobj(posterior['mass_2']):
        for keys in posterior:
            posterior[keys] = float(np.real(posterior[keys]))
    target_likelihood.parameters.update(posterior)
    reference_parameters = dict(luminosity_distance = priors2['luminosity_distance'],
                                geocent_time = priors2['geocent_time'])
    target_likelihood.parameters.update(reference_parameters)
    likelihood = target_likelihood.log_likelihood_ratio()
    print(samples['log_likelihood'].iloc[i],likelihood)

16:00 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_black_hole
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters
16:00 bilby INFO    : Loaded distance marginalisation lookup table from ../TD.npz.


59.806386915704934 59.80638686678582
60.270465592097025 60.2704640807553
60.32883868276258 60.32878740672286
60.33572764760174 60.335648783105476
61.37355782564555 61.373559460485346
61.44738926480932 61.44740283873642
61.88217383780284 61.8821756199675
61.89252575987484 61.8924702866146
61.89523999423545 61.894943525603566
62.02201167587191 62.022010927612435


In [6]:
import pickle

file_path = '/home/shunyin.cheung/other_PE_run/GW190924.json'
data_file = '/home/shunyin.cheung/other_PE_run/data/GW190924_data_dump.pickle'


result = bilby.core.result.read_in_result(file_path)
samples = result.posterior
args = process_bilby_result(result.meta_data['command_line_args'])
priors = result.priors
priors2 = copy.copy(priors)
psds=None

with open(data_file, 'rb') as f:
    data_dump = pickle.load(f)
try:
    ifo_list = data_dump.interferometers

except AttributeError:
    ifo_list = data_dump['ifo_list']

sampling_frequency = ifo_list.sampling_frequency
maximum_frequency = args['maximum_frequency']
minimum_frequency = args['minimum_frequency']
reference_frequency = args['reference_frequency']
roll_off = args['tukey_roll_off']
duration = args['duration']

waveform_name = args['waveform_approximant']
    

# test if bilby oscillatory waveform = gwmemory oscillatory waveform.
waveform_generator_osc = bilby.gw.waveform_generator.WaveformGenerator(
    duration=duration,
    sampling_frequency=sampling_frequency,
    frequency_domain_source_model= bilby.gw.source.lal_binary_black_hole,
    parameter_conversion = bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters,
    waveform_arguments=dict(duration=duration,
                            minimum_frequency=minimum_frequency,
                            maximum_frequency=maximum_frequency,
                            sampling_frequency=sampling_frequency,
                            reference_frequency=reference_frequency,
                            waveform_approximant=waveform_name,
                            )

)

target_likelihood = bilby.gw.likelihood.GravitationalWaveTransient(
    ifo_list,
    waveform_generator_osc,
    time_marginalization = True,
    distance_marginalization = True,
    distance_marginalization_lookup_table = '../TD.npz',
    jitter_time=True,
    priors = priors2,
    reference_frame = args['reference_frame'],
    time_reference = args['time_reference'],
)


for i in range(10):
    posterior = samples.iloc[i].to_dict()
    if np.iscomplexobj(posterior['mass_2']):
        for keys in posterior:
            posterior[keys] = float(np.real(posterior[keys]))
    target_likelihood.parameters.update(posterior)
    reference_parameters = dict(luminosity_distance = priors2['luminosity_distance'],
                                geocent_time = priors2['geocent_time'])
    target_likelihood.parameters.update(reference_parameters)
    likelihood = target_likelihood.log_likelihood_ratio()
    print(samples['log_likelihood'].iloc[i],likelihood)

16:21 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_black_hole
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters
16:21 bilby INFO    : Loaded distance marginalisation lookup table does not match for distance_array.
16:21 bilby INFO    : Building lookup table for distance marginalisation.
100%|██████████| 400/400 [00:47<00:00,  8.35it/s]


43.85931352246553 43.85930885748777
44.101655219565146 44.10168099912901
44.637883813644294 44.6378875924874
46.14454905100865 46.14454907146559
46.3715550492052 46.37147642925093
46.603375209029764 46.6033752087545
46.66206593840616 46.66206593591889
46.68458501494024 46.684587842259226
46.74755420727888 46.747636635602284
46.92007853859104 46.920075995563266
